In [1]:
import torch
import torchvision
import torch.optim as optim
from data.data import get_train_test_loader
from model.network import FastUpdateNet, TeacherNet
import torch.nn.functional as F
import time
import pickle

In [2]:
n_epochs = 4
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [3]:
train_loader, test_loader = get_train_test_loader('mnist')

In [4]:
train_losses = []
train_counter = []
test_losses = []
test_accuracies = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [5]:
def train_fastNet(epoch, network):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    # print(network.mNet.saver.grad)
    network.mNet.backwardHidden()
    correctness = (target == torch.argmax(output))
    optimizer.step()
    if batch_idx % log_interval == 0:
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))


def train_teacherNet(epoch, network):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    # print(network.mNet.saver.grad)
    correctness = (target == torch.argmax(output))
    optimizer.step()
    if batch_idx % log_interval == 0:
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))

In [6]:
def test(network):
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  test_accuracies.append(100. * correct / len(test_loader.dataset))
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [7]:
tNet = TeacherNet()
optimizer = optim.SGD(tNet.parameters(), lr=learning_rate, momentum=momentum)
test(tNet)
for epoch in range(1, n_epochs + 1):
  train_teacherNet(epoch, tNet)
  test(tNet)

/home/hao/Documents/magic-m/model/network.py:47: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
/home/hao/.local/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.3013, Accuracy: 1388/10000 (14%)


Test set: Avg. loss: 0.3622, Accuracy: 8947/10000 (89%)


Test set: Avg. loss: 0.2077, Accuracy: 9382/10000 (94%)


Test set: Avg. loss: 0.1572, Accuracy: 9524/10000 (95%)


Test set: Avg. loss: 0.1297, Accuracy: 9605/10000 (96%)



In [ ]:
torch.save(tNet, 'tNet.pt')

In [8]:
fNet = FastUpdateNet(teacherNet=tNet)
test(fNet)

tensor(10376246.)
tensor(27858688.)


/home/hao/Documents/magic-m/model/network.py:134: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(o_4)


tensor(27798928.)
tensor(2.4135e+08)
tensor(13467446.)
tensor(16670758.)
tensor(19288588.)
tensor(15150114.)
tensor(13950406.)
tensor(32791536.)

Test set: Avg. loss: 2997.4062, Accuracy: 1049/10000 (10%)



In [9]:
fNet = FastUpdateNet()
optimizer = optim.SGD(fNet.get_parameters(), lr=learning_rate, momentum=momentum)

with inv
error tensor(0.0003, grad_fn=<SumBackward0>)
0.018805503845214844

with pinv
error tensor(0.00004, grad_fn=<SumBackward0>)
0.02195906639099121

0.01661086082458496

In [10]:
train_losses = []
train_counter = []
test_losses = []
test_accuracies = []

test(fNet)
for epoch in range(1, n_epochs + 1):
  train(epoch, fNet)
  test(fNet)

tensor(332903.4375)
tensor(397599.6250)
tensor(6816679.)
tensor(374105.7188)
tensor(4019380.)
tensor(946502.8750)
tensor(1109840.5000)
tensor(477566.6250)
tensor(1243718.)
tensor(312749.3750)

Test set: Avg. loss: 34.4835, Accuracy: 1009/10000 (10%)



NameError: name 'train' is not defined

In [ ]:
# test_accuracies